In [6]:
import psycopg2
import config
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import getpass

hostname = 'localhost'
database='Test'
username= 'postgres'
pwd= getpass.getpass()
port_id='5432'

········


In [7]:
artist_data_df= pd.read_csv('../data/artist_details.csv',index_col=0)
tracks_data_df = pd.read_csv('../data/truncated_tracks.csv',index_col=0)
tracks_feature_df = pd.read_csv('../data/track_audio_features.csv',index_col=0)


In [8]:
tracks_data_df=tracks_data_df.loc[:,['artist_id','id','album_id','album_name','album_type','album_released_date','track_popularity']]
tracks_data_df['artist_id'] = tracks_data_df['artist_id'].str.replace('[','', regex=True)
tracks_data_df['artist_id'] = tracks_data_df['artist_id'].str.replace(']','', regex=True)
tracks_data_df['artist_id'] = tracks_data_df['artist_id'].str.replace("'","", regex=True)
tracks_data_df

,artist_id,id,album_id,album_name,album_type,album_released_date,track_popularity
0,74ASZWbe4lXaubB36ztrGX,3AhXZa8sUQht0UEdBJgpGc,6YabPKtZAjxwyWbuO9p4ZD,Highway 61 Revisited,album,1965-08-30,71
1,6olE6TJLqED3rqDCT0FyPh,5ghIJDpPoe3CfHMGu71E6T,2guirTSEqLizK7j9i1MTTZ,Nevermind (Remastered),album,1991-09-26,80
2,3WrFJ7ztbogyGnTHbHJFl2,0hKRSZhUGEhKU6aNSPBACZ,6QaVfG1pHYl1z15ZxkvVDW,Sgt. Pepper's Lonely Hearts Club Band (Remaste...,album,1967-06-01,66
3,3oDbviiivRWhXwIE8hxkVV,5Qt4Cc66g24QWwGP3YYV9y,3yGRRfowVUIj29zEwgLYY7,Smiley Smile (Mono & Stereo),album,1967-09-18,22
4,293zczrfYafIItmnmM3coR,2QfiRTz5Yc8DdShCxG1tB2,6eedtCtCjibu80yOhylSGL,Berry Is On Top,album,1959-07-01,76
...,...,...,...,...,...,...,...
9942,2vwI9jlKSgJbne3dlTzaLO,2QSD3K3b3BJ8DPhGhQfDPW,0uk9Hua2gX6jPT0O4jxjvC,Scared To Dance,album,1979-01-01,46
9943,7xTKLpo7UCzXSnlH7fOIoM,67mtN6BlsJR0LVyzvNZnGt,3FsCGJY0JqSxvgezoMWYzF,Whut? Thee Album,album,1992-01-01,64
9944,6nB0iY1cjSY1KyhYyuIIKH,1KcxkWlRSDNchLovHnHBKP,2RUX2ps4xMNTyI2iAwVyf3,M3LL155X,album,2015-08-14,41
9945,3LrsctPHK5wMdvEqvFN8BW,3h66NhIvR5cgi2xJlUUmgd,2Kwk8gZZk0cRjCuYaQ80jT,Happy Head,album,1986-10-27,13


In [9]:
tracks_feature_df = tracks_feature_df.iloc[:,:]
tracks_feature_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,0.557,95.263,audio_features,3AhXZa8sUQht0UEdBJgpGc,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4
1,0.502,0.912,1,-4.556,1,0.0564,0.000025,0.000173,0.1060,0.720,116.761,audio_features,5ghIJDpPoe3CfHMGu71E6T,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,https://api.spotify.com/v1/audio-analysis/5ghI...,301920,4
2,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,0.175,163.219,audio_features,0hKRSZhUGEhKU6aNSPBACZ,spotify:track:0hKRSZhUGEhKU6aNSPBACZ,https://api.spotify.com/v1/tracks/0hKRSZhUGEhK...,https://api.spotify.com/v1/audio-analysis/0hKR...,337413,4
3,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,0.331,133.574,audio_features,5Qt4Cc66g24QWwGP3YYV9y,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...,219147,4
4,0.534,0.803,10,-9.129,1,0.0743,0.741000,0.000061,0.3070,0.969,167.983,audio_features,2QfiRTz5Yc8DdShCxG1tB2,spotify:track:2QfiRTz5Yc8DdShCxG1tB2,https://api.spotify.com/v1/tracks/2QfiRTz5Yc8D...,https://api.spotify.com/v1/audio-analysis/2Qfi...,161560,4


In [10]:

tracks_feature_df=tracks_feature_df.loc[:,['id','type','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness'
,'liveness','valence','tempo','type','duration_ms','time_signature','uri','track_href','analysis_url']]


tracks_feature_df.head()

,id,type,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,uri,track_href,analysis_url
0,3AhXZa8sUQht0UEdBJgpGc,audio_features,0.482,0.721,0,-6.839,1,0.0321,0.731000,0.000000,0.1890,0.557,95.263,audio_features,369600,4,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...
1,5ghIJDpPoe3CfHMGu71E6T,audio_features,0.502,0.912,1,-4.556,1,0.0564,0.000025,0.000173,0.1060,0.720,116.761,audio_features,301920,4,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,https://api.spotify.com/v1/audio-analysis/5ghI...
2,0hKRSZhUGEhKU6aNSPBACZ,audio_features,0.364,0.457,4,-14.162,0,0.0675,0.290000,0.000106,0.9220,0.175,163.219,audio_features,337413,4,spotify:track:0hKRSZhUGEhKU6aNSPBACZ,https://api.spotify.com/v1/tracks/0hKRSZhUGEhK...,https://api.spotify.com/v1/audio-analysis/0hKR...
3,5Qt4Cc66g24QWwGP3YYV9y,audio_features,0.398,0.413,1,-10.934,1,0.0388,0.082200,0.000025,0.0891,0.331,133.574,audio_features,219147,4,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...
4,2QfiRTz5Yc8DdShCxG1tB2,audio_features,0.534,0.803,10,-9.129,1,0.0743,0.741000,0.000061,0.3070,0.969,167.983,audio_features,161560,4,spotify:track:2QfiRTz5Yc8DdShCxG1tB2,https://api.spotify.com/v1/tracks/2QfiRTz5Yc8D...,https://api.spotify.com/v1/audio-analysis/2Qfi...


In [12]:
conn_string = 'postgresql://{user}:{pwd}@{host}/{db}'.format(user=username,pwd=pwd,host=hostname,db=database)  
db = create_engine(conn_string)
conn = db.connect()

artist_data_df.to_sql('artist_details', con=conn, if_exists='replace',
          index=False)
tracks_data_df.to_sql('track_details', con=conn, if_exists='replace',
          index=False)  
tracks_feature_df.to_sql('track_features', con=conn, if_exists='replace',
          index=False)                    
conn = psycopg2.connect(conn_string
                        )
conn.autocommit = True
cursor = conn.cursor()
  



In [29]:
def get_top_15():
    """ query data from the vendors table """
    conn = None
    try:
        conn = psycopg2.connect(conn_string)
        cur = conn.cursor()
        cur.execute("select a.artist_names,a.track_id,b.album_name,b.album_released_date,b.track_popularity from artist_details a join track_details b on a.track_id = b.id order by 5 desc limit 15")
        print("The number of records: ", cur.rowcount)
        row = cur.fetchone()

        while row is not None:
            print(row)
            row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [39]:
print(get_top_15())

The number of records:  15
('Dua Lipa', '7ef4DlsgrMEH11cDZd32M6', 'One Kiss (with Dua Lipa)', '2018-04-06', 90)
('Blur', '7MXVkk9YMctZqd1Srtv4MB', 'Starboy', '2016-11-25', 90)
('Coldplay', '3AJwUDP919kvQ9QcozQPxg', 'Parachutes', '2000-07-10', 90)
('', '7lQ8MOhq6IN2w8EYcFNSUk', 'The Eminem Show', '2002-05-26', 89)
('Patsy Cline', '1DIXPcTDzTj8ZMHt3PDt8p', "Gangsta's Paradise", '1995-11-07', 89)
('Arctic Monkeys', '5FVd6KXrgO9B3JPmC8OPst', 'AM', '2013-09-09', 89)
('J Balvin', '0fea68AdmYNygeTGI4RC18', 'OASIS', '2019-06-28', 89)
('Adamski', '3sNVsP50132BTNlImLx70i', 'Yeezus', '2013-06-18', 89)
('Denzel Curry', '7fBv7CLKzipRk6EC6TWHOB', 'Beauty Behind The Madness', '2015-08-28', 88)
('Arctic Monkeys', '086myS9r57YsLbJpU0TgK9', 'AM', '2013-09-09', 88)
('Bruce Springsteen', '3yfqSUWxFvZELEM4PmlwIR', 'The Marshall Mathers LP', '2000-05-23', 88)
('Tears For Fears', '4RvWPyQ5RL0ao9LPZeSouE', 'Songs From The Big Chair (Super Deluxe Edition)', '1985-02-25', 88)
('Patsy Cline', '003vvx7Niy0yvhvHt4

In [30]:
conn.close()